In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
from sklearn.metrics.pairwise import haversine_distances
from math import radians

In [2]:
pref = 'basico'

In [3]:
df1 = pd.read_csv('reto_precios.csv')
df2 = pd.read_csv('basicos.csv', encoding='latin-1')

In [4]:
df2.head()

,id,nom_estab,raz_social,codigo_act,nombre_act,per_ocu,tipo_vial,nom_vial,tipo_v_e_1,nom_v_e_1,tipo_v_e_2,nom_v_e_2,tipo_v_e_3,nom_v_e_3,numero_ext,letra_ext,edificio,edificio_e,numero_int,letra_int,tipo_asent,nomb_asent,tipoCenCom,nom_CenCom,num_local,cod_postal,cve_ent,entidad,cve_mun,municipio,cve_loc,localidad,ageb,manzana,telefono,correoelec,www,tipoUniEco,latitud,longitud,fecha_alta
0,7461653,AGUA POTABLE,AGUA POTABLLE,221312,"Captación, tratamiento y suministro de agua re...",0 a 5 personas,CALLE,VENUSTIANO CARRANZA,PRIVADA,VENUSTIANO CARRANZA,CALLE,NUEVA,CALLE,EL JARABE,20.0,NaN,NaN,NaN,NaN,NaN,COLONIA,JESUS GOMEZ PORTUGAL (MARGARITAS),NaN,NaN,NaN,20909.0,1,AGUASCALIENTES,5,Jesús María,41,Jesús Gómez Portugal (Margaritas),0389,19,NaN,NaN,NaN,Fijo,21.998873,-102.291044,2019-11
1,27235,ASOCIACIÓN DE USUARIOS JUNTA DE AGUAS DEL DIST...,ASOCIACIÓN DE USUARIOS JUNTA DE AGUAS DEL DIST...,221311,"Captación, tratamiento y suministro de agua re...",11 a 30 personas,AVENIDA,JOSE VASCONCELOS,PROLONGACION,GUADALUPE DELGADO DE LIRA,AVENIDA,JOSE MARIA CHAVEZ,AVENIDA,PLUTARCO ELÍAS CALLES,4.0,B,NaN,NaN,NaN,NaN,COLONIA,CENTRO,NaN,NaN,NaN,20670.0,1,AGUASCALIENTES,6,Pabellón de Arteaga,1,Pabellón de Arteaga,0085,18,NaN,JUNTA_DE-AGUAS@YAHOO.COM,NaN,Fijo,22.147994,-102.281536,2010-07
2,39349,BIO GAS TECNOLOGY,COMISION FEDERAL DE ELECTRICIDAD,221119,Generación de electricidad a partir de otro ti...,0 a 5 personas,AVENIDA,UNIVERSIDAD,AVENIDA,AGUASCALIENTES NORTE,CALLE,SIERRA DE LA CANELA,CALLE,MONTE EVEREST,4.0,2,TORRE PLAZA BOSQUES,NaN,1001.0,NaN,FRACCIONAMIENTO,BOSQUES DEL PRADO,NaN,NaN,NaN,20127.0,1,AGUASCALIENTES,1,Aguascalientes,1,Aguascalientes,2259,19,NaN,NaN,NaN,Fijo,21.916732,-102.313583,2014-12
3,40194,CFE,COMISION FEDERAL DE ELECTRICIDAD,221111,Generación de electricidad a partir de combust...,0 a 5 personas,AVENIDA,CONVENCION DE 1914 NORTE,CALLE,CARLOS SAGREDO,AVENIDA,FUNDICION,CALLE,SAN FRANCISCO,2301.0,NaN,CENTRO Y PLAZA COMERCIAL,1.0,NaN,D 26,FRACCIONAMIENTO,SAN CAYETANO,CENTRO Y PLAZA COMERCIAL,PLAZA SAN MARCOS,D 26,20010.0,1,AGUASCALIENTES,1,Aguascalientes,1,Aguascalientes,0286,34,NaN,NaN,NaN,Fijo,21.895822,-102.310258,2014-12
4,38241,CFE,COMISION FEDERAL DE ELECTRICIDAD,221111,Generación de electricidad a partir de combust...,0 a 5 personas,CALZADA,JULIO CADENA,CALLE,BENIGNO CHAVEZ,CALLE,CALVARIO,CALZADA,EMILIANO ZAPATA,0.0,SN,NaN,NaN,0.0,NaN,COLONIA,ZONA CENTRO,NaN,NaN,NaN,20900.0,1,AGUASCALIENTES,5,Jesús María,1,Jesús María,0158,22,4499638929,NaN,NaN,Fijo,21.965489,-102.341126,2014-12


In [5]:
df2.columns

Index(['id', 'nom_estab', 'raz_social', 'codigo_act', 'nombre_act', 'per_ocu',
       'tipo_vial', 'nom_vial', 'tipo_v_e_1', 'nom_v_e_1', 'tipo_v_e_2',
       'nom_v_e_2', 'tipo_v_e_3', 'nom_v_e_3', 'numero_ext', 'letra_ext',
       'edificio', 'edificio_e', 'numero_int', 'letra_int', 'tipo_asent',
       'nomb_asent', 'tipoCenCom', 'nom_CenCom', 'num_local', 'cod_postal',
       'cve_ent', 'entidad', 'cve_mun', 'municipio', 'cve_loc', 'localidad',
       'ageb', 'manzana', 'telefono', 'correoelec', 'www', 'tipoUniEco',
       'latitud', 'longitud', 'fecha_alta'],
      dtype='object')

In [6]:
df1 = df1[['id','lat','lon']]
# Cambiar los nombres de las variables en df1 agregando sufijo 1
df1 = df1.rename(columns={ 'lat': 'lat1', 'lon': 'lon1'})
# Cambiar los nombres de las variables en df2 agregando sufijo 2
df2 = df2.rename(columns={'id': 'id2', 'latitud': 'lat2', 'longitud': 'lon2'})


In [7]:
# Definir la tolerancia espacial (en kilómetros)
tolerancia = 0.5

# Convertir las latitudes y longitudes a radianes
df1['lat_rad1'] = df1['lat1'].apply(radians)
df1['lon_rad1'] = df1['lon1'].apply(radians)
df2['lat_rad2'] = df2['lat2'].apply(radians)
df2['lon_rad2'] = df2['lon2'].apply(radians)

In [8]:
# Calcular la distancia haversine entre las coordenadas
distances = haversine_distances(df1[['lat_rad1', 'lon_rad1']], df2[['lat_rad2', 'lon_rad2']]) * 6371000/1000  # Convertir a kilómetros

# Encontrar las coincidencias dentro de la tolerancia
matches = distances <= tolerancia

In [9]:
matching_indices = np.where(matches)

# Crear una lista para almacenar los DataFrames combinados
combined_dfs = []

# Iterar sobre los índices y columnas de coincidencias
for idx, col in zip(*matching_indices):
    # Obtener los registros coincidentes de df1 y df2
    matching_record_df1 = df1.loc[idx]
    matching_record_df2 = df2.loc[col]
    
    # Combinar los registros en un DataFrame
    combined_record = pd.concat([matching_record_df1, matching_record_df2])
    combined_dfs.append(combined_record)

# Concatenar todos los DataFrames combinados en uno solo
final_combined_df = pd.concat(combined_dfs, axis=1).T

In [10]:
final_combined_df.head(3)

,id,lat1,lon1,lat_rad1,lon_rad1,id2,nom_estab,raz_social,codigo_act,nombre_act,per_ocu,tipo_vial,nom_vial,tipo_v_e_1,nom_v_e_1,tipo_v_e_2,nom_v_e_2,tipo_v_e_3,nom_v_e_3,numero_ext,letra_ext,edificio,edificio_e,numero_int,letra_int,tipo_asent,nomb_asent,tipoCenCom,nom_CenCom,num_local,cod_postal,cve_ent,entidad,cve_mun,municipio,cve_loc,localidad,ageb,manzana,telefono,correoelec,www,tipoUniEco,lat2,lon2,fecha_alta,lat_rad2,lon_rad2
0,5.0,19.419693,-99.156036,0.338938,-1.730599,6738050,SOLRA FOTOVOLTAICA 02 SA DE CV,SOLRA FOTOVOLTAICA 02 SA DE CV,221119,Generación de electricidad a partir de otro ti...,0 a 5 personas,EJE VIAL,1 PONIENTE (AVENIDA CUAUHTÉMOC),EJE VIAL,CHIHUAHUA,AVENIDA,ÁLVARO OBREGÓN,CALLE,FRONTERA,NaN,NaN,NaN,NaN,133.0,NaN,COLONIA,ROMA NORTE,NaN,NaN,NaN,6700.0,9,CIUDAD DE MÉXICO,15,Cuauhtémoc,1,Cuauhtémoc,1110,26,NaN,NaN,NaN,Fijo,19.418882,-99.154373,2016-10,0.338923,-1.73057
1,6.0,19.40302,-99.1687,0.338647,-1.73082,8458231,SISTEMA DE AGUA DE LA CIUDAD DE MÉXICO,AREA DE PITOMETRIA,221312,"Captación, tratamiento y suministro de agua re...",6 a 10 personas,EJE VIAL,4 SUR (RAFAEL DONDE),CALLE,PATRICIO SANZ,AVENIDA,COYOACAN,CALLE,PATRICIO SANZ,21.0,NaN,NaN,NaN,NaN,NaN,COLONIA,DE VALLE,NaN,NaN,NaN,3100.0,9,CIUDAD DE MÉXICO,14,Benito Juárez,1,Benito Juárez,0045,7,NaN,NaN,NaN,Fijo,19.399569,-99.16779,2019-11,0.338586,-1.730804
2,6.0,19.40302,-99.1687,0.338647,-1.73082,9069026,SISTEMA DE AGUAS DE LA CIUDAD DE MÉXICO,EL GOBIERNO DEL DF,221312,"Captación, tratamiento y suministro de agua re...",11 a 30 personas,EJE VIAL,4 SUR (RAFAEL DONDE),CALLE,PATRICIO SANZ,AVENIDA,COYOACAN,CALLE,PATRICIO SANZ,21.0,NaN,NaN,NaN,NaN,NaN,COLONIA,DE VALLE,NaN,NaN,NaN,3100.0,9,CIUDAD DE MÉXICO,14,Benito Juárez,1,Benito Juárez,0045,7,5555234378,NaN,NaN,Fijo,19.399569,-99.16779,2019-11,0.338586,-1.730804


In [11]:
len(final_combined_df)

191

In [12]:
final_combined_df['numero_ext'].value_counts()

21.0     129
505.0     10
0.0        2
14.0       2
250.0      2
8.0        1
Name: numero_ext, dtype: int64

In [13]:
final_combined_df['manzana'].value_counts()

7     129
26     35
14     10
5       8
6       2
15      2
1       2
10      2
12      1
Name: manzana, dtype: int64

In [14]:
final_combined_df['per_ocu'].value_counts()

11 a 30 personas      86
6 a 10 personas       44
0 a 5 personas        37
51 a 100 personas     10
31 a 50 personas      10
251 y más personas     4
Name: per_ocu, dtype: int64

In [15]:
personal_ocupado = {'0 a 5 personas' : 1,
'6 a 10 personas' : 2,
'11 a 30 personas' : 3,
'31 a 50 personas' : 4,
'51 a 100 personas' : 5,
'101 a 250 personas' : 6,
'251 y más personas' : 7}

final_combined_df['per_ocu'] = final_combined_df['per_ocu'].replace(personal_ocupado)
final_combined_df['per_ocu'].value_counts()

3    86
2    44
1    37
5    10
4    10
7     4
Name: per_ocu, dtype: int64

In [16]:
#per_ocu
#numero_ext
#manzana

In [17]:
final_combined_df['dif_lat_rad'] =final_combined_df['lat_rad1']-final_combined_df['lat_rad2']
final_combined_df['dif_lon_rad'] =final_combined_df['lon_rad1']-final_combined_df['lon_rad2']

In [18]:
(final_combined_df['lat_rad1']-final_combined_df['lat_rad2'])

0      0.000014
1       0.00006
2       0.00006
3       0.00006
4      0.000038
         ...   
186   -0.000069
187   -0.000041
188    0.000059
189    0.000059
190    0.000059
Length: 191, dtype: object

In [19]:
final_combined_df.entidad.value_counts()

CIUDAD DE MÉXICO    191
Name: entidad, dtype: int64

In [20]:
final_combined_df[final_combined_df.entidad.str.contains('NUEVO LEÓN')][['id','dif_lat_rad','dif_lon_rad']]

,id,dif_lat_rad,dif_lon_rad


In [21]:
final_combined_df[final_combined_df.entidad.str.contains('BAJA CALIFORNIA')][['id','dif_lat_rad','dif_lon_rad']]

,id,dif_lat_rad,dif_lon_rad


In [22]:
summary = final_combined_df[['id','per_ocu','numero_ext','manzana']]
summary.head()

,id,per_ocu,numero_ext,manzana
0,5.0,1,NaN,26
1,6.0,2,21.0,7
2,6.0,3,21.0,7
3,6.0,3,21.0,7
4,9.0,1,NaN,26


In [23]:
# Definir las columnas para la agrupación y agregación
aggregation_columns = ['per_ocu', 'numero_ext', 'manzana']  # Agrega aquí las columnas que quieras sumar

# Realizar la agrupación y las agregaciones
grouped_df = summary.groupby('id')[aggregation_columns].agg({'sum'}).reset_index()
grouped_df2 = summary.groupby('id').agg(total=('id', 'count')).reset_index()
grouped_df2.columns = ['id'] + [f'{pref}_total']



# Renombrar las columnas resultantes
grouped_df.columns = ['id'] + [f'{pref}_suma_{col}' for col in aggregation_columns]

# Mostrar el DataFrame resultante
centros = grouped_df2.merge(grouped_df)

In [24]:
centros.head()

,id,basico_total,basico_suma_per_ocu,basico_suma_numero_ext,basico_suma_manzana
0,5.0,1,1,0,26
1,6.0,3,8,63.0,21
2,9.0,1,1,0,26
3,10.0,1,1,0,26
4,32.0,1,1,0,26


In [25]:
centros.to_csv('centros_'+pref+'s'+'.csv')